In [66]:
import pandas as pd

In [67]:
df = pd.read_csv('lista_zawiadomień.csv', encoding='utf-8-sig', sep=";")

# Handle missing values in Nr zlec., Opis i Priorytet
df['Nr zlec.'] = df['Nr zlec.'].fillna('Brak danych')
df['Opis'] = df['Opis'].fillna('Brak opisu')
df['Priorytet'] = df['Priorytet'].fillna('Nieokreślony')

# Delete rows with NaN values in Oznaczenie (name of the machine)
df = df.dropna(subset=['Oznaczenie'])

# Delete not needed columns: Utworzone przez, Opis, Status systemu and Nr zlec.
df = df.drop(columns=['Utworzone przez', 'Opis', 'Status systemu', "Nr zlec."])

# Change type of the column Data zawiadom. to date
df['Data zawiadom.'] = pd.to_datetime(df['Data zawiadom.'], format='%d.%m.%Y', errors='coerce')

df

,Rodzaj zawiad.,Zawiadomienie,Data zawiadom.,Oznaczenie,Priorytet
0,1P,12708097,2025-06-30,1-Zespół pomp próżniowych BUSCH,4-niski
1,1P,12707957,2025-06-30,1-Trans. Sandwick za walcami 64M,Nieokreślony
2,1P,12707755,2025-06-30,1-Zbiornik olejowy 1m^3,Nieokreślony
3,1P,12707745,2025-06-30,1-Piec wafla górnego KB2,Nieokreślony
4,PQ,12707726,2025-06-30,1-Installations Other,3-średni
...,...,...,...,...,...
160648,PM,10143679,2014-04-01,2-Kinder Joy lini-Nalewarka goccia,Nieokreślony
160649,PM,10143678,2014-04-01,1-Kinder Joy lini-Modelator do K Joya,Nieokreślony
160650,PM,10143666,2014-04-01,1-Linia pakowania IFCR,Nieokreślony
160652,PM,10143508,2014-04-01,1-Tanks,Nieokreślony


In [68]:
from pathlib import Path
import pandas as pd

file_path = Path("urządzenie_struktura.txt")
with file_path.open(encoding="utf-8") as f:
    lines = [line.rstrip('\n') for line in f if line.strip()]

records = []
stack = []

for line in lines:
    level = len(line) - len(line.lstrip('\t'))
    name = line.strip()

    if level < len(stack):
        stack = stack[:level]
    elif level > len(stack):
        stack.extend([""] * (level - len(stack)))

    stack = stack[:level] + [name]

    record = {f"Level_{i+1}": val for i, val in enumerate(stack)}
    records.append(record)

df_hierarchy = pd.DataFrame(records)
df_hierarchy = df_hierarchy.drop(columns=["Level_1", "Level_2", "Level_3", "Level_5", "Level_11"])
if "Level_3" in df_hierarchy.columns:
    df_hierarchy = df_hierarchy[~df_hierarchy["Level_3"].str.contains("NIE UŻYWAĆ", na=False)]
    
# df_hierarchy = df_hierarchy.applymap(lambda x: x.replace('\t', '') if isinstance(x, str) else x)

# level_6_kod = df_hierarchy['Level_6'].str.extract(r'(\d{8})')
# df_hierarchy.insert(2, "Level_6_Kod", level_6_kod)
# df_hierarchy['Level_6'] = df_hierarchy['Level_6'].str.replace(r'\d{8}', '', regex=True)

# level_7_kod = df_hierarchy['Level_7'].str.extract(r'(\d{8})')
# df_hierarchy.insert(4, "Level_7_Kod", level_7_kod)
# df_hierarchy['Level_7'] = df_hierarchy['Level_7'].str.replace(r'\d{8}', '', regex=True)

# level_8_kod = df_hierarchy['Level_8'].str.extract(r'(\d{8})')
# df_hierarchy.insert(6, "Level_8_Kod", level_8_kod)
# df_hierarchy['Level_8'] = df_hierarchy['Level_8'].str.replace(r'\d{8}', '', regex=True)

# level_9_kod = df_hierarchy['Level_9'].str.extract(r'(\d{8})')
# df_hierarchy.insert(8, "Level_9_Kod", level_9_kod)
# df_hierarchy['Level_9'] = df_hierarchy['Level_9'].str.replace(r'\d{8}', '', regex=True)

# level_10_kod = df_hierarchy['Level_10'].str.extract(r'(\d{8})')
# df_hierarchy.insert(10, "Level_10_Kod", level_10_kod)
# df_hierarchy['Level_10'] = df_hierarchy['Level_10'].str.replace(r'\d{8}', '', regex=True)

# level_12_kod = df_hierarchy['Level_12'].str.extract(r'(\d{8})')
# df_hierarchy.insert(12, "Level_12_Kod", level_12_kod)
# df_hierarchy['Level_12'] = df_hierarchy['Level_12'].str.replace(r'\d{8}', '', regex=True)

df_hierarchy.to_csv("urządzenia_linie.csv", index=False, encoding="utf-8")

df_hierarchy

# for idx, row in df_hierarchy.iterrows():
#     if not (pd.isna(row["Level_12"]) or row["Level_12"] == ""):
#         print(row["Level_12"])


,Level_4,Level_6,Level_7,Level_8,Level_9,Level_10,Level_12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7125,PLPA-PR-UCL-076568\t\t\t\t\t\t\t\t\t\t\tManual...,10172793\t\t\t\t\t\t\t1-Owijarka palet BOSKY E...,NaN,NaN,NaN,NaN,NaN
7126,PLPA-PR-UCL-076568\t\t\t\t\t\t\t\t\t\t\tManual...,10241399\t\t\t\t\t\t\t1-Printer EBS Ink-Jet Sy...,NaN,NaN,NaN,NaN,NaN
7127,PLPA-PR-UCL-076568\t\t\t\t\t\t\t\t\t\t\tManual...,10241400\t\t\t\t\t\t\t1-Printer EBS Ink-Jet Sy...,NaN,NaN,NaN,NaN,NaN
7128,PLPA-PR-UCL-076568\t\t\t\t\t\t\t\t\t\t\tManual...,10241402\t\t\t\t\t\t\t1-Printer EBS Ink-Jet Sy...,NaN,NaN,NaN,NaN,NaN


In [69]:
for i in range(0, 100):
    print(df_hierarchy.iloc[i]["Level_4"])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-010703											RAFFAELLO LINIA 2
PLPA-PR-U11-0107

In [ ]:
u = {
    "U11": {"UGP11 RAFFAELLO": {
            {"010703": {"RAFFAELLO LINIA 2": {}}},
            {"010709": {"LINIA WAŻĄCO-PAKUJĄCA 12 T9 RAFFAELLO": {"506237"}}},
            }
            },
    "U12": {"UGP12 NUTELLA B-READY+KINDER CARDS": {}},
    "U13": {"UGP13 TABLETS+ROCHER FIGURES": {}},
    "U21": {"UGP21 K SURPRISE + FIGURA CAVA": {}},
    "U22": {"UGP22 KINDER JOY": {}},
    "U31": {"UGP31 NUTELLA + TOASTING": {}},
    "U32": {"UGP32 PREPARATION + DEOD.": {}},
    "U41": {"UGP41 KINDER BUENO MINI": {}},
    "U42": {"UGP42 KINDER BUENO WHITE": {}},
    "U43": {"UGP43 KINDER BUENO": {}},
    "UCL": {"CONTO LAVORO": {}}
}

'''Lokalizacja funkcjon PLPA-PR-U21-010123-002            Ważne od        09.07.25
Oznaczenie      FC MODELATOR AASTED

  PLPA-PR                        Ferrero Polska Produkcja
   |
   |--    PLPA-PR-001                    (NIE UŻYWAĆ) UGP1 RAF KJ
   |--    PLPA-PR-011                    (NIE UŻYWAĆ) UGP2 KS FC
   |--    PLPA-PR-021                    (NIE UŻYWAĆ) UGP3 PREP NUT TT
   |--    PLPA-PR-031                    (NIE UŻYWAĆ) UGP4 KB KBW
   |--    PLPA-PR-041                    (NIE UŻYWAĆ) CLA CUB STEL WORK
   |--    PLPA-PR-U05-011312             (NIE UŻYWAĆ) UGP5 B-READY, CARDS
   |--    PLPA-PR-U06                    (NIE UŻYWAĆ) UGP6 ROCHER, TABLETS
   |--    PLPA-PR-U11                    UGP11 RAFFAELLO
   |   |
   |   |--    PLPA-PR-U11-010703             RAFFAELLO LINIA 2
   |   |--    PLPA-PR-U11-010709             LINIA WAŻĄCO-PAKUJĄCA 12 T9 RAFFAELLO                              506237
   |   |--    PLPA-PR-U11-010716             LINIA WAŻĄCO-PAKUJĄCA 7 RAFFAELLO                                  506089
   |   |--    PLPA-PR-U11-010717             LINIA PAKUJĄCA ERMETICO RAFFAELLO                                  506238
   |   |--    PLPA-PR-U11-010718             LINIA WAŻĄCO-PAKUJĄCA 8 RAFFAELLO                                  506090
   |   |--    PLPA-PR-U11-010719             LINIA WAŻĄCO-PAKUJĄCA 10 T45 L.19 RAFFAE                           506091
   |   |--    PLPA-PR-U11-010721             LINIA WAŻĄCO-PAKUJĄCA 11 T8 L.21 RAFFAEL                           506092
   |   |---   PLPA-PR-U11-010796             LINIA ROZDRABNIANIA WIÓRKA                                         506016
   |
   |--    PLPA-PR-U12                    UGP12 NUTELLA B-READY+KINDER CARDS
   |   |
   |   |--    PLPA-PR-U12-010801             KINDER CARDS
   |   |--    PLPA-PR-U12-011312             NUTELLA B-READY
   |
   |--    PLPA-PR-U13                    UGP13 TABLETS+ROCHER FIGURES
   |   |
X  |   |--    PLPA-PR-U13-010175             IFCR PAKOWNIE WIEWIÓRKA/BAŁWAN/JAJKO                               506169
   |   |--    PLPA-PR-U13-010176             FCR PAKOWANIE ZAJĄC G100 GR7                                       506264
   |   |--    PLPA-PR-U13-010177             FC ROCHER(MODELATOR&PAKOWANIE)NEW
   |   |--    PLPA-PR-U13-010178             FC ROCHER(MODELATOR&PAKOWANIE)OLD
   |   |--    PLPA-PR-U13-010401             TABLETS
   |
   |--    PLPA-PR-U21                    UGP21 K SURPRISE + FIGURA CAVA
   |   |
   |   |--    PLPA-PR-U21-010123             FIGURA CAVA
   |   |--    PLPA-PR-U21-010199             KS DOZOWNIKI ODRZYWOŁEK                                            506026
   |   |--    PLPA-PR-U21-010121             KINDER SURPRISE
   |
   |--    PLPA-PR-U22                    UGP22 KINDER JOY
   |   |
   |   |--    PLPA-PR-U22-010320             KJ MODELOWANIE TARTUFINO
   |   |--    PLPA-PR-U22-010325             KJ LINIA 1
   |   |--    PLPA-PR-U22-010326             KJ LINIA 2
   |   |--    PLPA-PR-U22-010335             KJ T3 VALIGETTA                                                    506385
   |   |--    PLPA-PR-U22-010337             KJ T3 SCHUBERT                                                     506285
   |   |--    PLPA-PR-U22-019329             KJ TERMOFORMATURA
   |
   |--    PLPA-PR-U31                    UGP31 NUTELLA + TOASTING
   |   |
   |   |--    PLPA-PR-U31-011330             Nutella T200 glass
   |   |--    PLPA-PR-U31-011331             Nutella T400
   |   |--    PLPA-PR-U31-011332             Nutella T750
   |   |--    PLPA-PR-U31-012012             Palarnia orzechów 1 Buhler                                         506012
   |   |--    PLPA-PR-U31-012013             Palarnia orzechów 4 Brovind                                        506112
   |   |--    PLPA-PR-U31-012016             Palarnia orzechów 3 Mini                                           506013
   |   |--    PLPA-PR-U31-012112             Palarnia orzechów 2 Barth                                          506312
   |   |--    PLPA-PR-U31-012120             Rozdrabnianie kakao                                                506317
   |
   |--    PLPA-PR-U32                    UGP32 PREPARATION + DEOD.
   |   |
   |   |--    PLPA-PR-U32-012010             PREP. Kremu RAF                                                    506010
   |   |--    PLPA-PR-U32-012011             PREP. Kremu KB
   |   |--    PLPA-PR-U32-012015             Dezodorator 1                                                      506015
   |   |--    PLPA-PR-U32-012017             Pasta kokosowa RAF                                                 506017
   |   |--    PLPA-PR-U32-012018             PREP. Kremu FC KS                                                  506018
   |   |--    PLPA-PR-U32-012020             TOPIELNIKI PREP1                                                   506014
   |   |---   PLPA-PR-U32-012021             KBD enrobing
   |   |--    PLPA-PR-U32-012110             Prep2 Krem KB                                                      506310
   |   |--    PLPA-PR-U32-012111             PREP. Kremu NUT
   |   |--    PLPA-PR-U32-012113             PREP. KREM KBW                                                     506211
   |   |--    PLPA-PR-U32-012115             Dezodorator 2                                                      506315
   |   |--    PLPA-PR-U32-012118             FC Mild and Dark choc. mixing                                      506313
   |   |--    PLPA-PR-U32-012119             KB Rework                                                          506319
   |   |--    PLPA-PR-U32-012210             Jasny krem (TAV/K CARDS/KJ)                                        506400
   |   |--    PLPA-PR-U32-012211             Ciemny krem (K CARDS/KJ)                                           506401
   |   |--    PLPA-PR-U32-012215             Dezodoryzator 3                                                    506415
   |   |--    PLPA-PR-U32-012220             TOPIELNIKI PREP3                                                   506314
   |
   |--    PLPA-PR-U41                    UGP41 KINDER BUENO MINI
   |   |
   |   |--    PLPA-PR-U41-011441             KB MINI 1.
   |   |--    PLPA-PR-U41-011540             KB MINI 2. Modelator-nowy
   |   |--    PLPA-PR-U41-011541             KB MINI 2. Modelator(OBLEWANIE)+PAKOWANI
   |   |--    PLPA-PR-U41-011542             KB MINI 2. Pakowanie MIX                                           506278
   |
   |--    PLPA-PR-U42                    UGP42 KINDER BUENO WHITE
   |   |
   |   |--    PLPA-PR-U42-011442             KBW 1/COCO
   |   |--    PLPA-PR-U42-011443             KBW 2.
   |   |--    PLPA-PR-U42-011646             KBW 1. Mastello                                                    506473
   |   |--    PLPA-PR-U42-011647             KBW 2. Carton Case                                                 506474
   |
   |--    PLPA-PR-U43                    UGP43 KINDER BUENO
   |   |
   |   |--    PLPA-PR-U43-011444             KB / KBW Isola Multipack                                           506378
   |   |--    PLPA-PR-U43-011445             KB 2.
   |   |--    PLPA-PR-U43-011446             KB 2. Pakowanie T6                                                 506075
   |   |--    PLPA-PR-U43-011447             KB 2. Carton Case                                                  506373
   |   |--    PLPA-PR-U43-011544             KB Multipack 2(KB 3)                                               506178
   |   |--    PLPA-PR-U43-011545             KB 3.
   |
   |--    PLPA-PR-UCL                    CONTO LAVORO
       |
       |--    PLPA-PR-UCL-026565             Mix 1/4 PALLETT
       |--    PLPA-PR-UCL-026567             A CAL G234                                                         506289
       |--    PLPA-PR-UCL-026568             Flowpack Cards T2x12                                               506289
       |--    PLPA-PR-UCL-026569             Linia Kalendarzy KS T24/ G329                                      506095
       |--    PLPA-PR-UCL-026570             Manual line 1 - PELUCHE                                            506095
       |---   PLPA-PR-UCL-026571             Manual line 2 - PELUCHE                                            506095
       |--    PLPA-PR-UCL-026572             Manual line - KJ / KS                                              506095
       |---   PLPA-PR-UCL-026573             Linia BUNNY HOUSE 155G EASTER XXX                                  506095
       |--    PLPA-PR-UCL-026574             Manual line 5 - TIC TAC                                            506095
       |--    PLPA-PR-UCL-026575             TINS LINE
       |--    PLPA-PR-UCL-026578             CAL BUENO G167                                                     506289
       |--    PLPA-PR-UCL-026590             Pralines STELLA                                                    506096
       |--    PLPA-PR-UCL-026592             Pralines CUBOTTO                                                   506097
       |--    PLPA-PR-UCL-026594             CAL DBS T27                                                        506096
       |--    PLPA-PR-UCL-026595             Manual line  (DBS T8/RAF TUBO)                                     506096
       |---   PLPA-PR-UCL-026597             Manual line cal. dibesten domek XXX                                506096
       |--    PLPA-PR-UCL-026598             PRALINES HANGABLE TINS                                             506096
       |--    PLPA-PR-UCL-026696             K MIX G34 TIN                                                      506096
       |--    PLPA-PR-UCL-026699             CAL KINDER TRAIN G221                                              506095
       |--    PLPA-PR-UCL-036565             Manual production KS-Logi Pack                                     506395
       |--    PLPA-PR-UCL-056565             Manual co-packing Belsk                                            506195
       |---   PLPA-PR-UCL-066565             Manual co-packing Tavolette                                        506193
       |--    PLPA-PR-UCL-066566             Mixing toys                                                        506191
       |--    PLPA-PR-UCL-066567             Manual co-packing Odrzywolek                                       506193
       |--    PLPA-PR-UCL-066568             Kinderino                                                          506193
       |--    PLPA-PR-UCL-066569             Kinder Joy T2 box                                                  506227
       |--    PLPA-PR-UCL-066570             Snack CAL295                                                       506193
       |--    PLPA-PR-UCL-076566             Manual co-packing Lub CAL 149/203                                  506192
       |--    PLPA-PR-UCL-076567             Manual co-packing Lub CAL 351/236                                  506192
       |--    PLPA-PR-UCL-076568             Manual co-packing Lub CAL404                                       506192
       |---   PLPA-PR-UCL-076569             Manual co-packing Lub Mix bag                                      506194
'''